<a href="https://colab.research.google.com/github/cmartinez4269/DS-2002/blob/main/DS2002__Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dean Paler (hdh4ys) & Carina Martinez (fyv5hu)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import Statements
import requests
import pandas as pd
from io import StringIO
import json
import sqlite3

CSV/JSON to User Output Preference

In [14]:
# Function to get number of columns and rows in a dataframe
def column_records_df(df):
    num_cols = len(df.columns)
    num_recs = len(df)
    print("Number of columns: " + str(num_cols))
    print("Number of records: " + str(num_recs))

# Fetching CSV file data
def fetch_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as err:
        print(f"An error occurred: {err}")

# Fetching api JSON data
def fetch_api_data(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()
        return data["results"]
    except Exception as err:
        print(f"An error occurred: {err}")  # Handle other errors

# Altering DF
def alter_csv_data(df):
  # Adding 'Free Time' column to see what time isn't spent studying, sleeping, or working out
  df['Free_Time'] = 24 - (df['Study_Hours'] + df['Sleep_Duration'] + (df['Physical_Activity']/60))

  # If negative, then there is no free time in a 24 hour period
  df["Negative_Free_Time_Flag"] = df["Free_Time"] < 0

  #Dropping unnecessary columsn
  df.drop(columns=['Study_Hours', 'Weekday_Sleep_End', 'Weekend_Sleep_End', 'Weekday_Sleep_Start', 'Weekend_Sleep_Start'], inplace = True)

# Altering DF
def alter_df(df):
  #Removing all of the columns except for ID, name, powerstats data, and connection.
  filtered_df = df[['id', 'name', 'powerstats.intelligence', 'powerstats.strength', 'powerstats.speed', 'powerstats.durability', 'powerstats.power', 'powerstats.combat', 'connections.group-affiliation', 'connections.relatives']]
  return filtered_df

def save_output(df, format):

    #Outputting df to json
    if format == 'json':
      file_name = input('Enter the output file name (with extension): ')
      df.to_json(file_name, orient='records')
      print(f'JSON data saved to {file_name}')

    # Outputting df to csv file
    elif format == 'csv':
      file_name = input('Enter the output file name (with extension): ')
      df.to_csv(file_name, index=False)
      print(f'CSV data saved to {file_name}')

    # Outputting df to sql database
    elif format == 'sql':
      db_name = input('Enter the SQL database name (add.sql at the end): ')
      table_name = input('Enter name for table: ')
      conn = sqlite3.connect (db_name)
      for column in df.columns:
        if df[column].dtype not in [str, int, float]:
          df[column] = df[column].astype(str)
          if table_name:
              df.to_sql(table_name, conn, if_exists='replace', index=False)
      conn.close()
      print(f'Data stored in SQL database: {db_name}, table: {table_name}')
      column_records_df(df)
    else:
      print('Invalid output format. Supported formats are: json, csv, sql')

# Converting data based on the given input
def process_data(input_format, data):
    if input_format == 'json':
      return pd.json_normalize(data)
    elif input_format == 'csv':
      return pd.read_csv(StringIO(data))

def main():
  # api url and file path
  api_url = "https://superheroapi.com/api/98d918463db4cba596fc90cb20983844/search/batman"
  file_path = "/content/student_sleep_patterns.csv"

  #input format from user
  input_format = input('Enter input format (csv, json): ').lower()

  #if json input, then the api data will be fetched, else the csv file will be used
  if input_format == 'json':
    batman_data = fetch_api_data(api_url)
    df = process_data(input_format, batman_data)
  elif input_format == 'csv':
    df = fetch_csv_data(file_path)

  #pre processing records and columns numbers
  print('Pre Processing Summary:')
  column_records_df(df)

  # processing the df based on the type of input
  if input_format == 'json':
    df = alter_df(df)
  elif input_format == 'csv':
    alter_csv_data(df)

  #post processing reocrds and columns numbers
  print('Post Processing Summary:')
  column_records_df(df)

  #Asking user for type of output and saving it
  output_format = input('Enter output format (csv, json, sql): ').lower()
  save_output(df, output_format)

if __name__ == "__main__":
  main()



Enter input format (csv, json): json
Pre Processing Summary:
Number of columns: 26
Number of records: 3
Post Processing Summary:
Number of columns: 10
Number of records: 3
Enter output format (csv, json, sql): csv
Enter the output file name (with extension): csv.csv
CSV data saved to csv.csv
